In [2]:
! pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 72.7 MB/s eta 0:00:00


In [47]:
def constant_oracle(n):
    """
    when f(x) = 0 or 1 (constant) , then for input xy ,
    output wil be xy (same)
    """
    oracle = QuantumCircuit(n + 1)
    # oracle.x(n)  # for f(x)=1
    return oracle

def balanced_oracle(n):
    """
    when f(x) is balanced , then outputs will half opposite,
    so just applying operatios y xor f(x) , for all x
    """
    oracle = QuantumCircuit(n + 1)
    for i in range(n):
        oracle.cx(i, n)
    return oracle

In [46]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def deutsch_jozsa(f_oracle, n):
    # n+1 bits(input) and n for classical bit(output)
    qc = QuantumCircuit(n + 1, n)

    # Step 1: Setting up y
    qc.x(n)
    qc.h(n)

    # Step 2: Hadamard transform to input x
    for i in range(n):
        qc.h(i)

    qc.barrier()

    # Step 3: Append the oraclef f(x)
    qc = qc.compose(f_oracle, qubits=range(n + 1))

    qc.barrier()

    # Step 4: Apply Hadamard again to input qubits
    for i in range(n):
        qc.h(i)

    # Step 5: Measure input qubits
    for i in range(n):
        qc.measure(i, i)

    return qc

In [53]:
# Choose number of input bits
n = 3
# Preparing Uf
oracle = constant_oracle(n)

In [54]:
dj_circuit = deutsch_jozsa(oracle, n)
dj_circuit.draw()

┌───┐      ░  ░ ┌───┐┌─┐      
q_0: ┤ H ├──────░──░─┤ H ├┤M├──────
     ├───┤      ░  ░ ├───┤└╥┘┌─┐   
q_1: ┤ H ├──────░──░─┤ H ├─╫─┤M├───
     ├───┤      ░  ░ ├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├──────░──░─┤ H ├─╫──╫─┤M├
     ├───┤┌───┐ ░  ░ └───┘ ║  ║ └╥┘
q_3: ┤ X ├┤ H ├─░──░───────╫──╫──╫─
     └───┘└───┘ ░  ░       ║  ║  ║ 
c: 3/══════════════════════╩══╩══╩═
                           0  1  2

In [55]:
sim = Aer.get_backend('aer_simulator')
result = sim.run(dj_circuit).result()
f = result.get_counts()
f

{'000': 1024}